# Compute mean of observation for a specific year

In [1]:
# modules importation
from trends_functions import *
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
# run observations
varss = ['od550aer', 'od550lt1aer', 'od550gt1aer', 'ang4487aer', 'concpm25', 'concpm10', 'concso4', 'scatc550dryaer', 'absc550aer']
regions = ['EUROPE', 'NAMERICA', 'SAMERICA', 'NAFRICA', 'SAFRICA', 'ASIA', 'AUSTRALIA']

In [3]:
OBS_MAP = {}
for var in varss:
    # computation parameters
    params = get_params()
    params['kind'] = 'obs'
    params['min_dobs'] = 300
    fill_params(params, var)
    params['var'] = var
    
    obs_source = params['source']
    reader = pya.io.ReadUngridded(obs_source)
    obs_data = reader.read(vars_to_retrieve=var)
    
    print('compute trend')
    _, OBS_MAP[var], _ = process_trend(
        obs_data, params,
        plot=False, show_plot=False, save_plot=False, write_json=False
    )

compute trend
no flags
no flags EUROPE station:  ZvenigoroddsEi_GCWD
no flags ASIA station:  Zhongshan_UnivityDh ASIA station:  DRAGON_NIER
no flags NAMERICA station:  Yumaowknife_AuroraC NAMERICA station:  DRAGON_BLLRT
no flags SAMERICA station:  Veracruz_MAX_MEXFOP
no flags NAFRICA station:  Zinder_DMNporttutea Tuz_Golu_3
region:  SAFRICA station:  Walvis_Bay_airport
Less than 7 points in selected period
no flags
no flags AUSTRALIA station:  Tinga_Tinganand_QLD
compute trendD station:  ZvenigorodportityaP WORLD station:  Calipso_Kennedyvill WORLD station:  Chapada WORLD station:  DRAGON_Mt_Rokko WORLD station:  DRAGON_UiTM WORLD station:  Vienna_UNIVIE
no flags
no flags EUROPE station:  ZvenigorodMOW1erCWD
no flags ASIA station:  Zhongshan_UnivityDh ASIA station:  DRAGON_Mt_Rokko ASIA station:  Liangning ASIA station:  Saih_Salam ASIA station:  Umm_Al_Quwain
no flags NAMERICA station:  Yumaowknife_AuroraCCalipso_Sterling_PO
no flags SAMERICA station:  UdeConcepcion-CEFOP
no flags NAF

## put this in a dataframe

In [4]:
#'concpm25', 'concpm10', 'concso4', 'scatc550dryaer', 'absc550aer'

In [9]:
thead = ['']+regions
tbody = []

for var in varss:
    if var in ['concpm25', 'concpm10', 'scatc550dryaer', 'absc550aer']:
        fmt = '{:2.1f}'
    else:
        fmt = '{:3.2f}'
    print(var)
    fill_params(params, var)
    unit = params['unit']
    if unit == '':
        str_unit = ''
    else:
        str_unit = ' ('+unit+')'
    line = [params['ylabel']+str_unit] + [fmt.format(OBS_MAP[var][region]['trends']['trends']['2000-2014']['data'][0]) if region in OBS_MAP[var] and 'trends' in OBS_MAP[var][region]['trends'] and '2000-2014' in OBS_MAP[var][region]['trends']['trends'] else '-' for region in regions]
    tbody.append(line)

    df = pd.DataFrame(tbody, columns=thead)

od550aer
od550lt1aer
od550gt1aer
ang4487aer
concpm25
concpm10
concso4
scatc550dryaer
absc550aer


In [11]:
with pd.option_context("max_colwidth", 1000):
    latable = df.to_latex(index=False, longtable=False, escape=False)
    #replace rule with hline for ACP
    latable = latable.replace('toprule','tophline')
    latable = latable.replace('midrule','middlehline')
    latable = latable.replace('bottomrule','bottomhline')
    print(latable)

\begin{tabular}{llllllll}
\tophline
                    & EUROPE & NAMERICA & SAMERICA & NAFRICA & SAFRICA &  ASIA & AUSTRALIA \\
\middlehline
                AOD &   0.17 &     0.10 &     0.15 &    0.26 &       - &  0.35 &      0.10 \\
            AOD<1µm &   0.14 &     0.08 &     0.12 &    0.11 &       - &  0.18 &      0.05 \\
            AOD>1µm &   0.03 &     0.02 &     0.03 &    0.10 &       - &  0.11 &      0.03 \\
                 AE &   1.44 &     1.46 &     1.30 &    0.72 &       - &  1.06 &      0.97 \\
     PM2.5 (µg.m-3) &   12.7 &      6.0 &        - &       - &       - &     - &         - \\
      PM10 (µg.m-3) &   16.8 &     11.5 &        - &    19.6 &       - &     - &         - \\
       SO4 (µg.m-3) &   2.01 &     1.45 &        - &    2.98 &       - &  1.97 &         - \\
 Scat. Coef. (1/Mm) &   29.6 &     23.8 &        - &       - &       - &     - &         - \\
  Abs. Coef. (1/Mm) &    2.0 &      2.3 &        - &       - &       - &     - &         - \\
\bottomhlin